In [1]:
# useing this variable for saving models and summaries
exp_name = "CNN-LSTM4004"

## Running CNN then RNN
5 layers CNN [2, 2, 2, 2, 2], then RNN, bias_shape = [32,64,64, 64, 64]

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import sys
from hdf5storage import savemat
import pickle

sys.path.append('../../../src')
sys.path.append('../../../src/models')

In [3]:
import models.CNN_lstm_model as CNN_lstm_model
import training  
import evaluation.nn_eval as nn_eval 

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

## Model

In [5]:
## Model Parameter
n_variables= 12
learning_rate = 0.001
batch_size = 10

pooling_stride = [32, 1, 1, 1, 1, 1]
weights_shape = [[9,n_variables, 32],
                 [9, 32, 32],
                 [9, 32, 32],
                 [9, 32, 32],
                 [9, 32, 32],
                 [9, 32, 32]]

bias_shape = [32, 32, 32, 32, 32, 32, 32]
activation = tf.nn.relu

lstm_n_hidden = 128
n_classes = 1


#positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [6]:
pos_weight

3.186018221237171

In [7]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [8]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 


In [9]:
model = CNN_lstm_model.Model(n_variables, n_classes)

In [10]:
model.build_cnn_lstm_net(pooling_stride,
                         weights_shape,
                         bias_shape,
                         activation,
                         lstm_n_hidden,
                         n_classes,
                         learning_rate,
                         pos_weight)

In [ ]:
train_obj = training.Trainer(model,
                 train_data, train_gt,
                 validation_data, validation_gt,
                 sess,
                 summary_path,
                 batch_size,
                 global_step = 0)

In [ ]:
train_obj.train(max_iter = 10000,
                train_eval_step=100,
                validation_eval_step=100,
                display_step=100)

Tarin Iter 0, auc= 0.292028, fmeasure_max= 0.412482, precision= 0.287755, recall= 0.728055
Train Random guess:  , auc= 0.224123fmeasure_max= 0.370687, precision= 0.229082, recall= 0.970740
Validation Iter 0, auc= 0.246566, fmeasure_max= 0.401000, precision= 0.282992, recall= 0.687822
Validation Random guess:  auc= 0.239272, fmeasure_max= 0.372365, precision= 0.229600, recall= 0.984563
Minibatch Loss= 1.116195
Tarin Iter 100, auc= 0.701782, fmeasure_max= 0.740806, precision= 0.690049, recall= 0.799622
Train Random guess:  , auc= 0.203000fmeasure_max= 0.342728, precision= 0.206802, recall= 1.000000
Validation Iter 100, auc= 0.664019, fmeasure_max= 0.699078, precision= 0.683607, recall= 0.715266
Validation Random guess:  auc= 0.224212, fmeasure_max= 0.371659, precision= 0.228787, recall= 0.989708
Minibatch Loss= 0.713449
Tarin Iter 200, auc= 0.847372, fmeasure_max= 0.787763, precision= 0.830645, recall= 0.749091
Train Random guess:  , auc= 0.230829fmeasure_max= 0.354414, precision= 0.2164

In [ ]:
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=train_obj.global_step)

In [ ]:
saver = tf.train.Saver()
saver.restore(sess, checkpoint_path+'-1000')

In [ ]:
probability_of_pos = nn_eval.evaluate_model(model,
                                   sess,
                                   test_data,
                                   test_gt)

In [15]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [16]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})